# Author:
## Zashank Khare

#### Creating Spark Session

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Linear_Regression').getOrCreate()

#### Importing important libraries

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter('ignore')

from pyspark.ml.linalg import Vectors     
# Vectors class is used to represent dense and sparse vectors in Spark.

from pyspark.ml.feature import VectorAssembler  
# It is used in Spark ML pipelines, where it helps prepare data for ML algorithms that expect a single vector column as input.

from pyspark.ml.regression import LinearRegression

#### Loading datasets

In [3]:
df = spark.read.csv('Ecommerce_Customers.csv', header = True, inferSchema = True)

#### First five rows of df

In [4]:
df.show(5)

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [5]:
for x in df.head(1)[0]:
    print(x)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


#### Schema of df

In [6]:
df.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [7]:
print('No. of records present in the df: ',df.count())

No. of records present in the df:  500


#### Descriptive Statistics

In [8]:
from pyspark.sql.functions import round, col
stats = df.summary()
stats.select(stats['summary'],*[round(stats[c],2).alias(c) for c in stats.columns if c != 'summary']).show()

+-------+-----+-------+------+------------------+-----------+---------------+--------------------+-------------------+
|summary|Email|Address|Avatar|Avg Session Length|Time on App|Time on Website|Length of Membership|Yearly Amount Spent|
+-------+-----+-------+------+------------------+-----------+---------------+--------------------+-------------------+
|  count|500.0|  500.0| 500.0|             500.0|      500.0|          500.0|               500.0|              500.0|
|   mean| null|   null|  null|             33.05|      12.05|          37.06|                3.53|             499.31|
| stddev| null|   null|  null|              0.99|       0.99|           1.01|                 1.0|              79.31|
|    min| null|   null|  null|             29.53|       8.51|          33.91|                0.27|             256.67|
|    25%| null|   null|  null|             32.34|      11.39|          36.34|                2.93|             444.97|
|    50%| null|   null|  null|             33.08

In [9]:
df.distinct().count()

500

In [10]:
df.select('Avatar').distinct().count()

138

In [11]:
print('Showing the average of all the numerical features per Avatar:')
for c in ['Avg Session Length','Time on App','Time on Website','Length of Membership','Yearly Amount Spent']:
    var = ''
    var = 'avg'+'('+ c + ')'
    view1 = df.select('Avatar',col(c)).groupBy('Avatar').mean()
    view1 = view1.select('Avatar',round(col(var),2).alias(c)).orderBy(c, ascending = False)
    print(view1.show(5))

Showing the average of all the numerical features per Avatar:
+------------+------------------+
|      Avatar|Avg Session Length|
+------------+------------------+
|   GoldenRod|             34.46|
|       Azure|             34.31|
|MidnightBlue|             34.26|
|        Gray|             34.25|
|   MistyRose|             34.19|
+------------+------------------+
only showing top 5 rows

None
+-------------+-----------+
|       Avatar|Time on App|
+-------------+-----------+
|        Coral|      13.33|
|DarkSlateGray|      13.33|
|         Snow|      13.25|
|   Chartreuse|      13.23|
|         Peru|      13.22|
+-------------+-----------+
only showing top 5 rows

None
+-----------+---------------+
|     Avatar|Time on Website|
+-----------+---------------+
|    SkyBlue|          39.03|
|      Black|          38.56|
|NavajoWhite|          38.44|
| DarkSalmon|          38.31|
|ForestGreen|           38.3|
+-----------+---------------+
only showing top 5 rows

None
+--------------+----

#### Model and Fit

In [12]:
df.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [13]:
Cols = ['Avg Session Length','Time on App','Time on Website','Length of Membership']

In [14]:
assembler = VectorAssembler(inputCols = Cols, outputCol='features')
assem_data = assembler.transform(df)

In [15]:
assem_data.head()

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))

In [16]:
final_data = assem_data.select('features','Yearly Amount Spent')

In [17]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [18]:
train, validation = final_data.randomSplit([.8,.2])

In [19]:
train.summary().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                391|
|   mean| 502.22364911796933|
| stddev|  77.09618916281244|
|    min|  275.9184206503857|
|    25%| 447.18764430541853|
|    50%|   502.132789227254|
|    75%|  549.9041461052942|
|    max|  765.5184619388373|
+-------+-------------------+



In [20]:
validation.summary().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                109|
|   mean| 488.87681031348075|
| stddev|   86.3804883095363|
|    min| 256.67058229005585|
|    25%| 438.30370784642975|
|    50%|  490.6004425003995|
|    75%|  544.4092721605869|
|    max|  744.2218671047146|
+-------+-------------------+



In [21]:
Regressor = LinearRegression(labelCol= 'Yearly Amount Spent')
model = Regressor.fit(train)

In [22]:
validation_result = model.evaluate(validation)

In [23]:
validation_result.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-16.241752625208278|
|   5.68818208308619|
|-12.863101263217175|
| -7.270684076394446|
| 3.9995738077938086|
| 4.1790169994786766|
|-14.422936841326873|
|  17.16052531256861|
|  7.273956957900225|
| -6.315049865125445|
|-1.7902791290238156|
|-10.031412918696674|
|  -8.36304308710777|
| 5.9823242973410515|
|  5.773154072747502|
| 18.561041411034466|
|  5.717282946316914|
| 1.1296002820740796|
|-0.5812894622952172|
|  5.770891949232521|
+-------------------+
only showing top 20 rows



In [24]:
validation_result.rootMeanSquaredError

9.78965779216199

In [25]:
# Models performs good as the RMSE value is acceptable as comparison to the mean, std of the target variable.

In [26]:
df.select('Yearly Amount Spent').summary().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    25%| 444.96655165329264|
|    50%|   498.635598489176|
|    75%|  549.1315732872171|
|    max|  765.5184619388373|
+-------+-------------------+



In [27]:
validation_result.meanAbsoluteError

7.72498227706742

In [28]:
validation_result.meanSquaredError

95.83739968763797

In [29]:
validation_result.r2

0.987036972135358

In [30]:
# R2 value is also good as it saying the model explains the 98.3% of the variances of the data

## Thank You
#### Happy Learning